<h1>Neural network Modeling</h1>

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import seaborn as sns
import yfinance as yf
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
#from torchviz import make_dot
from tqdm.notebook import tqdm
from datetime import timedelta
import matplotlib.pyplot as plt
#from torchsummary import summary
from IPython.display import Image
import plotly.graph_objects as go
from IPython.display import Image, display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import warnings
import os
warnings.filterwarnings("ignore")
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import ray.cloudpickle as pickle
import tempfile
from functools import partial
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import gc, random, optuna
import matplotlib.pyplot as plt
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from sklearn.metrics import mean_absolute_error
import inspect
from hcr_nn.models import HCRCond2D, build_hcr_cond2d
from hcr_nn.basis import select_basis

c:\Users\X\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# --- Imports & path setup -------------------------------------------------
import os, sys, math, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from typing import Callable

np.random.seed(42); random.seed(42); torch.manual_seed(42)

# Make sure we can import hcr_nn whether we run from repo root or examples/
CWD = os.path.abspath(os.getcwd())
REPO_ROOT = os.path.abspath(os.path.join(CWD, '..')) if os.path.basename(CWD) == 'examples' else CWD
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

sys.path.insert(1, '/path/to/application/app/folder')

from hcr_nn.layers import CDFNorm
from hcr_nn.models import build_hcr_cond2d, HCRCond2D
from hcr_nn.basis import select_basis
from hcr_nn.density import conditional_density, expected_u1_given_u2
from hcr_nn.neuron import HCRNeuron

print('hcr_nn imported. Repo root =', REPO_ROOT)

hcr_nn imported. Repo root = c:\Users\X\Desktop\basically all\HCRNN\HCR-NN-Library\paper_stuffs\poc_stock_market


In [4]:
import random, numpy as np, torch
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [14]:
module_path = os.path.abspath(os.path.join('../hcr_nn/layers.py'))
if module_path not in sys.path:
    sys.path.append(module_path)

from hcr_nn.layers import *

<H2>Gathering data of stock tickers</H2>

In [1]:
import glob, pandas as pd, os, re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

path_multiple_companies = '../paper_data/djia29'
csv_files = glob.glob(os.path.join(path_multiple_companies, "*.csv"))

#stuff to put before the main loop begins
columns = ['close', 'open', 'volume', 'high', 'low']
scores = []
prediction_accuracies = []
names = []
#names.append(file.split('.')[0])
for x in csv_files:
    #loading dataset
    df = pd.read_csv(x)

    #changing type of column data
    df['date'] = pd.to_datetime(df['date'])
    df['volume'] = df['volume'].astype('float64')
    result = re.search(r'[^\\]+$', x).group()
    names.append(result)


print(names)

c:\Users\X\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


['AAPL.csv', 'AXP.csv', 'BA.csv', 'CAT.csv', 'CSCO.csv', 'CVX.csv', 'DIS.csv', 'GS.csv', 'HD.csv', 'IBM.csv', 'INTC.csv', 'JNJ.csv', 'JPM1.csv', 'KO.csv', 'MCD.csv', 'MMM.csv', 'MSFT.csv', 'NKE.csv', 'PFE.csv', 'PG.csv', 'TRV.csv', 'UNH.csv', 'UTX.csv', 'V.csv', 'VZ.csv', 'WBA.csv', 'WMT.csv', 'XOM.csv']


In [2]:
import glob

path_multiple_companies = '../paper_data/djia29'
csv_files = glob.glob(os.path.join(path_multiple_companies, "*.csv"))
companies_data = {}

for x in csv_files:
    
    #loading dataset
    df = pd.read_csv(x)

    #changing type of column data
    df['date'] = pd.to_datetime(df['date'])
    df['volume'] = df['volume'].astype('float64')

    df5 = df.copy(deep=True)
    df5 = df5.reset_index()
    def get_tech_ind(data):
        data['MA5'] = data.iloc[:,4].rolling(window=7).mean() #Close column
        data['MA10'] = data.iloc[:,4].rolling(window=10).mean() #Close Column

        data['MACD'] = data.iloc[:,4].ewm(span=26).mean() - data.iloc[:,1].ewm(span=12,adjust=False).mean()
        #This is the difference of Closing price and Opening Price

        # Create Bands
        data['8SD'] = data.iloc[:, 4].rolling(8).std()
        data['upper_band'] = data['MA10'] + (data['8SD'] * 2)
        data['lower_band'] = data['MA10'] - (data['8SD'] * 2)

        # Create Exponential moving average
        data['EMA'] = data.iloc[:,4].ewm(com=0.5).mean()

        return data

    tech_df = get_tech_ind(df5)
    dataset = tech_df.iloc[10:,:].reset_index(drop=True)
    dataset.head()
    df5['Date'] = pd.to_datetime(df5['Date'], utc=True)
    def tech_ind(dataset):
        fig,ax = plt.subplots(figsize=(15, 8), dpi = 200)
        x_ = range(3, dataset.shape[0])
        x_ = list(dataset.index)


    tech_ind(df5)
    dataset_5_days = df5.iloc[6:,:].reset_index(drop=True)
    dataset_5_days.head()
    dataset_10_days = df5.iloc[11:,:].reset_index(drop=True)
    dataset_10_days.head()
    dataset_5_days['Daily_Return'] = dataset_5_days['Close'].pct_change()

    # jeśli nie ma trade_sign → utwórz
    if 'trade_sign' not in dataset_5_days.columns:
        # wariant 1: znak jednodniowej zmiany ceny Close
        ds = dataset_5_days.copy()
        close = ds['Close'].astype(float)
        trade_sign = np.sign(close.diff()).fillna(0.0)  # -1, 0, 1
        ds['trade_sign'] = trade_sign
        dataset_5_days = ds.dropna().copy()

    dataset_5_days_trade_sign = dataset_5_days[['open', 'close', 'high', 'low', 'trade_sign']]
    # normalize 5 days data
    scaler = MinMaxScaler(feature_range=(-1,1)).fit(dataset_5_days_trade_sign)

    data_5_days_trade_sign = scaler.transform(dataset_5_days_trade_sign)
    seq_len_5_days=6
    sequences_5_days_trade_sign=[]
    for index in range(len(data_5_days_trade_sign) - seq_len_5_days + 1): 
        sequences_5_days_trade_sign.append(data_5_days_trade_sign[index: index + seq_len_5_days])
    sequences_5_days_trade_sign = np.array(sequences_5_days_trade_sign)

    sequences_5_days_trade_sign
    valid_set_size_percentage_trade_sign = 10 
    test_set_size_percentage_trade_sign = 10 

    valid_set_size_trade_sign = int(np.round(valid_set_size_percentage_trade_sign/100*sequences_5_days_trade_sign.shape[0]))  
    test_set_size_trade_sign  = int(np.round(test_set_size_percentage_trade_sign/100*sequences_5_days_trade_sign.shape[0]))
    train_set_size_trade_sign = sequences_5_days_trade_sign.shape[0] - (valid_set_size_trade_sign + test_set_size_trade_sign)

    x_train_5_days_trade_sign = sequences_5_days_trade_sign[:train_set_size_trade_sign,:-1,:]
    y_train_5_days_trade_sign = sequences_5_days_trade_sign[:train_set_size_trade_sign,-1,:]
        
    x_valid_5_days_trade_sign = sequences_5_days_trade_sign[train_set_size_trade_sign:train_set_size_trade_sign+valid_set_size_trade_sign,:-1,:]
    y_valid_5_days_trade_sign = sequences_5_days_trade_sign[train_set_size_trade_sign:train_set_size_trade_sign+valid_set_size_trade_sign,-1,:]
        
    x_test_5_days_trade_sign = sequences_5_days_trade_sign[train_set_size_trade_sign+valid_set_size_trade_sign:,:-1,:]
    y_test_5_days_trade_sign = sequences_5_days_trade_sign[train_set_size_trade_sign+valid_set_size_trade_sign:,-1,:]
    
    from torch.utils.data import Dataset, DataLoader
    import torch

    class SequenceDataset(Dataset):
        """
        Gwarantuje spójne kształty:
        x: (N, T-1, F)
        y: (N, F)
        """
        def __init__(self, x_np, y_np):
            x = torch.as_tensor(x_np, dtype=torch.float32)
            y = torch.as_tensor(y_np, dtype=torch.float32)

            # Jeśli ktoś przez pomyłkę poda (N,1,F) albo (N,T,F) -> weź ostatni krok
            if y.ndim == 3:
                y = y[:, -1, :]

            assert x.ndim == 3, f"Expected x.ndim==3, got {x.ndim}"
            assert y.ndim == 2, f"Expected y.ndim==2, got {y.ndim}"
            assert x.shape[0] == y.shape[0], "x and y must have same N"
            assert x.shape[2] == y.shape[1], "feature dim must match (F)"

            self.x = x
            self.y = y

        def __len__(self):
            return self.x.shape[0]

        def __getitem__(self, idx):
            return self.x[idx], self.y[idx]


    def make_loaders(
        x_train, y_train,
        x_valid, y_valid,
        x_test,  y_test,
        batch_size: int = 32,
        shuffle_train: bool = True,
        num_workers: int = 0,
        pin_memory: bool = False,
    ):
        train_ds = SequenceDataset(x_train, y_train)
        valid_ds = SequenceDataset(x_valid, y_valid)
        test_ds  = SequenceDataset(x_test,  y_test)

        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=shuffle_train,
                            num_workers=num_workers, pin_memory=pin_memory, drop_last=False)
        valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=pin_memory, drop_last=False)
        test_dl  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=pin_memory, drop_last=False)
        return train_dl, valid_dl, test_dl


    # --- zbuduj dataloadery z już przygotowanych tablic ---
    train_dataloader_5_days_trade_sign, \
    valid_dataloader_5_days_trade_sign, \
    test_dataloader_5_days_trade_sign = make_loaders(
        x_train_5_days_trade_sign, y_train_5_days_trade_sign,
        x_valid_5_days_trade_sign, y_valid_5_days_trade_sign,
        x_test_5_days_trade_sign,  y_test_5_days_trade_sign,
        batch_size=32, shuffle_train=True, num_workers=0, pin_memory=False
    )

    # Kontrola kształtów
    xb, yb = next(iter(train_dataloader_5_days_trade_sign))
    print("Batch shapes:", tuple(xb.shape), tuple(yb.shape))  # (B, T-1, F), (B, F)

    import torch.nn.functional as F  # opcjonalnie
    from torch import nn

    class NeuralNetwork(nn.Module):
        """
        Wejście:  x -> (B, T, F)
        Wyjście:  y -> (B, F)   (predykcja ostatniego kroku)
        """
        def __init__(self, num_feature: int, hidden_size: int = 64, num_layers: int = 1, dropout: float = 0.0):
            super().__init__()
            self.lstm = nn.LSTM(
                input_size=num_feature,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0.0
            )
            self.head = nn.Linear(hidden_size, num_feature)

        def forward(self, x):
            # x: (B, T, F)
            output, (hidden, cell) = self.lstm(x)
            last_hidden = hidden[-1]         # (B, hidden_size)
            out = self.head(last_hidden)     # (B, F)
            return out

    model = NeuralNetwork(5)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    #from torchinfo import summary
    #summary(model, input_size=(32, 5, 5))  # (batch, seq_len, features)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    mse = nn.MSELoss()
    
    def train(dataloader, device="cpu"):
        model.train()
        running_loss = 0.0
        n_batches = 0

        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            pred = model(x)           # (B, F)
            loss = mse(pred, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            n_batches += 1

        return running_loss / max(n_batches, 1)

    def evaluate(dataloader, device="cpu"):
        model.eval()
        running_loss = 0.0
        n_batches = 0

        with torch.no_grad():
            for x, y in dataloader:
                x = x.to(device)
                y = y.to(device)
                pred = model(x)        # (B, F)
                loss = mse(pred, y)
                running_loss += loss.item()
                n_batches += 1

        return running_loss / max(n_batches, 1)

    def plot_loss(train_losses, valid_losses):
        plt.plot(train_losses, color='black', label='test loss')
        plt.plot(valid_losses, color='blue', label='validation loss')

        plt.title('Loss function comparison')
        plt.xlabel('time')
        plt.ylabel('loss values')
        plt.legend(loc='best')
        plt.show()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    n_epochs = 1300
    patience = 50
    best_valid_loss = float('inf')
    epochs_no_improve = 0

    train_losses = []
    valid_losses = []

    for epoch_instance in range(1, n_epochs + 1):
        train_loss = train(train_dataloader_5_days_trade_sign, device=device)
        valid_loss = evaluate(valid_dataloader_5_days_trade_sign, device=device)

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        improved = valid_loss < best_valid_loss - 1e-8
        if improved:
            best_valid_loss = valid_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'saved_weights.pt')  # zapisuj state_dict
        else:
            epochs_no_improve += 1

        if epoch_instance % 10 == 0 or improved:
            print(f"Epoch {epoch_instance:4d} | Train: {train_loss:.6f} | Val: {valid_loss:.6f}")

        if epochs_no_improve >= patience:
            print(f"Early stop (patience={patience}) @ epoch {epoch_instance}")
            break

    plot_loss(train_losses=train_losses, valid_losses=valid_losses)
    # Ładujemy model i przełączamy w eval — UWAGA: ładujemy state_dict, a nie pełny obiekt!
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_test = NeuralNetwork(5).to(device)
    state = torch.load('saved_weights.pt', map_location=device)
    model_test.load_state_dict(state)
    model_test.eval()

    @torch.no_grad()
    def predict_full(model, loader, device="cpu"):
        ys, ps = [], []
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            pb = model(xb)           # (B, F)
            ys.append(yb.cpu())
            ps.append(pb.cpu())
        y = torch.cat(ys, dim=0).numpy()    # (N_test, F)
        p = torch.cat(ps, dim=0).numpy()    # (N_test, F)
        return y, p

    y_test_np, y_pred_np = predict_full(model_test, test_dataloader_5_days_trade_sign, device=device)
    print("Prediction shapes:", y_test_np.shape, y_pred_np.shape)

    # Rysujemy np. 'Close' = idx=1 (Open=0, Close=1, High=2, Low=3, trade_sign=4)
    idx = 1
    n_train = len(y_train_5_days_trade_sign)
    x_axis = np.arange(n_train, n_train + y_test_np.shape[0])

    plt.figure(figsize=(9,4))
    plt.plot(x_axis, y_test_np[:, idx],  color='black', label='test target')
    plt.plot(x_axis, y_pred_np[:, idx],  color='green', label='test prediction')
    plt.title('future stock prices')
    plt.xlabel('time [days]')
    plt.ylabel('normalized price')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

    print("MAE  (Close):", mean_absolute_error(y_test_np[:, idx], y_pred_np[:, idx]))
    print("MAPE (Close):", mean_absolute_percentage_error(y_test_np[:, idx], y_pred_np[:, idx]))

    ## New after Jarek sugestions
    # === KONFIG ===
    HORIZONS = [1, 3, 5, 10]     # w krokach (dni / interwały jak w Twoich danych)
    INPUT_LEN = 32               # długość okna wejściowego (możesz zwiększyć)
    TEST_SIZE = 0.15
    VALID_SIZE = 0.15
    BATCH_SIZE = 64
    LR = 1e-3
    EPOCHS = 500
    PATIENCE = 40
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    MH_FEATURES = ['open', 'close', 'high', 'low', 'trade_sign']

    # Seed dla powtarzalności
    def set_seed(seed=42):
        random.seed(seed); np.random.seed(seed)
        torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    set_seed(42)

    # Bierzemy dane z istniejącej ramki; jeśli nie masz 'dataset_5_days', użyj tej, którą masz po "New".
    if 'dataset_5_days' in globals():
        df_src = dataset_5_days.copy()
    else:
        # Jeśli w notatniku masz inną ramkę z tymi samymi kolumnami, podmień tutaj:
        df_src = dataset_5_days_trade_sign.copy()

    # Upewnij się, że kolumny są
    missing = [c for c in MH_FEATURES if c not in df_src.columns]
    if missing:
        raise ValueError(f"Brakuje kolumn w danych: {missing}. Podmień MH_FEATURES lub źródło danych.")

    # Skalowanie na osobnym skalerze, żeby nie popsuć wcześniejszych rzeczy
    scaler_mh = MinMaxScaler(feature_range=(-1, 1))
    X_full_np = scaler_mh.fit_transform(df_src[MH_FEATURES].values)   # shape: (N_all, F)

    def build_multi_horizon_sequences(x_full, input_len, horizons):
        """
        Zwraca:
        X: (N, input_len, F)
        Y: (N, H, F)  — gdzie H = len(horizons)
        """
        F = x_full.shape[1]
        H = len(horizons)
        max_h = max(horizons)
        seqs_X, seqs_Y = [], []

        for t in range(input_len, len(x_full) - max_h):
            x_win = x_full[t - input_len:t]                       # (input_len, F)
            y_targets = []
            for h in horizons:
                y_targets.append(x_full[t + h, :])               # (F,)
            y_targets = np.stack(y_targets, axis=0)               # (H, F)
            seqs_X.append(x_win)
            seqs_Y.append(y_targets)

        X = np.stack(seqs_X, axis=0)   # (N, input_len, F)
        Y = np.stack(seqs_Y, axis=0)   # (N, H, F)
        return X, Y

    X_mh, Y_mh = build_multi_horizon_sequences(X_full_np, INPUT_LEN, HORIZONS)

    N = X_mh.shape[0]
    n_test  = int(round(N * TEST_SIZE))
    n_valid = int(round(N * VALID_SIZE))
    n_train = N - n_test - n_valid
    assert n_train > 0 and n_valid > 0 and n_test > 0, "Za mało danych na taki podział."

    X_train_mh = X_mh[:n_train]
    Y_train_mh = Y_mh[:n_train]
    X_valid_mh = X_mh[n_train:n_train+n_valid]
    Y_valid_mh = Y_mh[n_train:n_train+n_valid]
    X_test_mh  = X_mh[n_train+n_valid:]
    Y_test_mh  = Y_mh[n_train+n_valid:]

    X_train_mh.shape, Y_train_mh.shape


    # --- Parametry HCR ---
    FEATURE_NAME = "Close"
    BASIS_NAME   = "polynomial"
    DEGREE       = 16
    GRID_SIZE    = 512
    LR_HCR       = 1e-3
    EPOCHS_HCR   = 800
    PATIENCE_HCR = 134
    EPS_DENS     = 1e-3

    assert FEATURE_NAME in MH_FEATURES, f"{FEATURE_NAME=} nie ma w MH_FEATURES={MH_FEATURES}"
    f_idx   = MH_FEATURES.index(FEATURE_NAME)
    h_idx   = 0  # H=1 => indeks 0


    def to_u(x_scaled):  # [-1,1] -> [0,1]
        return (x_scaled + 1.0) / 2.0

    u2_all = X_mh[:, -1, f_idx]
    u1_all = Y_mh[:, h_idx, f_idx]   # target H=1
    u2_all_u = to_u(u2_all)
    u1_all_u = to_u(u1_all)


    N = u2_all_u.shape[0]
    n_test  = int(round(N * TEST_SIZE))
    n_valid = int(round(N * VALID_SIZE))
    n_train = N - n_test - n_valid

    u2_tr, u2_va, u2_te = np.split(u2_all_u, [n_train, n_train+n_valid])
    u1_tr, u1_va, u1_te = np.split(u1_all_u, [n_train, n_train+n_valid])


    class HCRDataset(torch.utils.data.Dataset):
        def __init__(self, u2, u1):
            self.x = torch.tensor(np.stack([u1, u2], axis=1), dtype=torch.float32)
            self.y = torch.tensor(u1, dtype=torch.float32)
        def __len__(self): return self.x.shape[0]
        def __getitem__(self, i): return self.x[i], self.y[i]

    dl_tr = torch.utils.data.DataLoader(HCRDataset(u2_tr, u1_tr), batch_size=512, shuffle=True)
    dl_va = torch.utils.data.DataLoader(HCRDataset(u2_va, u1_va), batch_size=1024, shuffle=False)
    dl_te = torch.utils.data.DataLoader(HCRDataset(u2_te, u1_te), batch_size=1024, shuffle=False)

    hcr = build_hcr_cond2d(
        degree=DEGREE,
        basis=BASIS_NAME,
        grid_size=GRID_SIZE,
        coeff_init="xavier",
        dtype=torch.float32,
        device=DEVICE,
    ).to(DEVICE)

    opt = torch.optim.Adam(hcr.parameters(), lr=LR_HCR, weight_decay=1e-3)
    loss_fn = nn.MSELoss()

    best_va = float("inf")
    bad = 0
    hist_hcr = {"train": [], "valid": []}

    for ep in range(1, EPOCHS_HCR+1):
        # train
        hcr.train()
        bl = []
        for xb, yb in dl_tr:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            pred = hcr(xb)         # (B,) — E[u1|u2] w [0,1]
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            bl.append(loss.item())
        tr = float(np.mean(bl))

        # valid
        hcr.eval()
        with torch.no_grad():
            vl = []
            for xb, yb in dl_va:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                vl.append(loss_fn(hcr(xb), yb).item())
            va = float(np.mean(vl))

        hist_hcr["train"].append(tr); hist_hcr["valid"].append(va)
        improved = va < best_va - 1e-9
        if improved:
            best_va = va; bad = 0
            best_state = {k: v.detach().cpu().clone() for k,v in hcr.state_dict().items()}
        else:
            bad += 1

        if ep % 20 == 0 or improved:
            print(f"[HCR {ep:4d}] train={tr:.6f} | valid={va:.6f} | best={best_va:.6f}")
        if bad >= PATIENCE_HCR:
            print(f"HCR Early stop @ {ep} (patience={PATIENCE_HCR})")
            break

    if 'best_state' in locals():
        hcr.load_state_dict(best_state)

    @torch.no_grad()
    def predict_hcr(loader):
        hcr.eval()
        Ps, Ys = [], []
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            Ps.append(hcr(xb).cpu())
            Ys.append(yb.cpu())
        p = torch.cat(Ps).numpy()
        y = torch.cat(Ys).numpy()

        return 2.0*p - 1.0, 2.0*y - 1.0

    u1_pred_scaled, u1_true_scaled = predict_hcr(dl_te)

    def mape_safe(y_true, y_pred, eps=1e-8):
        y_true = np.asarray(y_true).ravel()
        y_pred = np.asarray(y_pred).ravel()
        denom = np.maximum(np.abs(y_true), eps)
        return float(np.mean(np.abs((y_true - y_pred)/denom)))

    mae_hcr  = mean_absolute_error(u1_true_scaled, u1_pred_scaled)
    mape_hcr = mape_safe(u1_true_scaled, u1_pred_scaled)
    print(f"[HCR] {FEATURE_NAME} | MAE={mae_hcr:.6f} | MAPE={mape_hcr:.6f}")

    # --- Wykres HCR vs target  ---
    N_test_len   = u1_true_scaled.shape[0]
    n_train_old  = len(y_train_5_days_trade_sign) if 'y_train_5_days_trade_sign' in globals() else 0
    x_axis       = np.arange(n_train_old, n_train_old + N_test_len)

    plt.figure(figsize=(10,4))
    plt.plot(x_axis, u1_true_scaled,  label='Target (H=1)', lw=2.0, color='black')
    plt.plot(x_axis, u1_pred_scaled,  label=f'HCR ({BASIS_NAME}, deg={DEGREE}, grid={GRID_SIZE})', lw=1.4)
    plt.title(f'{FEATURE_NAME} — HCR vs target (scaled)')
    plt.xlabel('Time index'); plt.ylabel('scaled value (-1..1)')
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

    # import optuna
    # from sklearn.metrics import mean_absolute_error
    #
    # def objective(trial):
    #
    #     lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    #     weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True)
    #     patience = trial.suggest_int("patience", 10, 500)
    #
    #     hcr = build_hcr_cond2d(
    #         degree=DEGREE,
    #         basis=BASIS_NAME,
    #         grid_size=GRID_SIZE,
    #         coeff_init="xavier",
    #         dtype=torch.float32,
    #         device=DEVICE,
    #     ).to(DEVICE)
    #
    #     opt = torch.optim.Adam(hcr.parameters(), lr=lr, weight_decay=weight_decay)
    #     loss_fn = nn.MSELoss()
    #
    #     best_va = float("inf")
    #     bad = 0
    #     for ep in range(1, EPOCHS_HCR+1):
    #         # --- train
    #         hcr.train()
    #         bl = []
    #         for xb, yb in dl_tr:
    #             xb, yb = xb.to(DEVICE), yb.to(DEVICE)
    #             opt.zero_grad()
    #             pred = hcr(xb)
    #             loss = loss_fn(pred, yb)
    #             loss.backward()
    #             opt.step()
    #             bl.append(loss.item())
    #         tr = float(np.mean(bl))
    #
    #         # --- valid
    #         hcr.eval()
    #         with torch.no_grad():
    #             vl = []
    #             for xb, yb in dl_va:
    #                 xb, yb = xb.to(DEVICE), yb.to(DEVICE)
    #                 vl.append(loss_fn(hcr(xb), yb).item())
    #             va = float(np.mean(vl))
    #
    #         if va < best_va - 1e-9:
    #             best_va = va; bad = 0
    #             best_state = {k: v.detach().cpu().clone() for k,v in hcr.state_dict().items()}
    #         else:
    #             bad += 1
    #
    #         if bad >= patience:
    #             break
    #
    #     if 'best_state' in locals():
    #         hcr.load_state_dict(best_state)
    #
    #     @torch.no_grad()
    #     def predict_hcr(loader):
    #         hcr.eval()
    #         Ps, Ys = [], []
    #         for xb, yb in loader:
    #             xb = xb.to(DEVICE)
    #             Ps.append(hcr(xb).cpu())
    #             Ys.append(yb.cpu())
    #         p = torch.cat(Ps).numpy()
    #         y = torch.cat(Ys).numpy()
    #         return 2.0*p - 1.0, 2.0*y - 1.0
    #
    #     u1_pred_scaled, u1_true_scaled = predict_hcr(dl_te)
    #
    #     def mape_safe(y_true, y_pred, eps=1e-8):
    #         y_true = np.asarray(y_true).ravel()
    #         y_pred = np.asarray(y_pred).ravel()
    #         denom = np.maximum(np.abs(y_true), eps)
    #         return float(np.mean(np.abs((y_true - y_pred)/denom)))
    #
    #     mae_hcr  = mean_absolute_error(u1_true_scaled, u1_pred_scaled)
    #     mape_hcr = mape_safe(u1_true_scaled, u1_pred_scaled)
    #     print(f"[HCR] {FEATURE_NAME} | MAE={mae_hcr:.6f} | MAPE={mape_hcr:.6f}")
    #
    #     return mae_hcr
    #
    # study = optuna.create_study(direction='minimize')
    # study.optimize(objective, n_trials=500)
    # print("Best Hyperparameters:", study.best_trial.params)

    # --- Optuna pod HCR  ---

    # ----------------------- Pomocnicze -----------------------

    def set_trial_seed(base, tnum):
        seed = int(base + tnum)
        random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        return seed

    @torch.no_grad()
    def _predict_scaled(model, loader, device):
        """Zwraca (pred, true) w skali [-1,1]"""
        model.eval()
        Ps, Ys = [], []
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True)
            Ps.append(model(xb).cpu())
            Ys.append(yb.cpu())
        p = torch.cat(Ps).numpy()
        y = torch.cat(Ys).numpy()
        return 2.0 * p - 1.0, 2.0 * y - 1.0


    # ----------------------- Objective -----------------------

    def objective(trial):
        # ---- seed na trial
        set_trial_seed(1337, trial.number)

        # ---- Hiperparametry (SZERSZE zakresy/ mozna jeszcze ulepszyć)
        degree     = trial.suggest_int("degree", 2, 48, step=1)
        grid_size  = trial.suggest_categorical("grid_size", [128, 256, 512, 1024, 2048, 4096])
        lr         = trial.suggest_float("lr", 1e-6, 5e-2, log=True)
        weight_dec = trial.suggest_float("weight_decay", 1e-8, 1e-1, log=True)
        patience   = trial.suggest_int("patience", 500, min(1000, EPOCHS_HCR))
        clip_grad  = trial.suggest_categorical("clip_grad", [0.0, 0.5, 1.0, 2.0])
        opt_kind   = trial.suggest_categorical("optimizer", ["adam", "adamw"])

        # ---- Model
        hcr = build_hcr_cond2d(
            degree=degree,
            basis=BASIS_NAME,
            grid_size=grid_size,
            coeff_init="xavier",
            dtype=torch.float32,
            device=DEVICE,
        ).to(DEVICE)

        # ---- Opt/los/scheduler
        if opt_kind == "adam":
            opt = torch.optim.Adam(hcr.parameters(), lr=lr, weight_decay=weight_dec)
        else:
            opt = torch.optim.AdamW(hcr.parameters(), lr=lr, weight_decay=weight_dec)

        loss_fn = nn.MSELoss()  # trenowanie dalej na MSE w [0,1]
        sched   = torch.optim.lr_scheduler.ReduceLROnPlateau(
            opt, mode="min", factor=0.5, patience=max(3, patience // 4)
        )

        best_mae = float("inf")
        bad = 0
        best_state = None
        min_delta = 1e-4  # wymagamy sensownej poprawy na MAE

        for ep in range(1, EPOCHS_HCR + 1):
            # --- train
            hcr.train()
            for xb, yb in dl_tr:
                xb, yb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True)
                opt.zero_grad(set_to_none=True)
                pred = hcr(xb)           # [0,1]
                loss = loss_fn(pred, yb) # MSE na [0,1]
                loss.backward()
                if clip_grad and clip_grad > 0.0:
                    torch.nn.utils.clip_grad_norm_(hcr.parameters(), max_norm=clip_grad)
                opt.step()

            # --- valid (jednym przebiegiem liczymy MAE[-1,1])
            p_va, y_va = _predict_scaled(hcr, dl_va, DEVICE)
            va_mae = mean_absolute_error(y_va.ravel(), p_va.ravel())

            # raport do Optuny + pruning
            trial.report(va_mae, step=ep)
            if trial.should_prune():
                raise optuna.TrialPruned()

            # scheduler + early stopping po MAE
            sched.step(va_mae)
            improved = (best_mae - va_mae) > min_delta
            if improved:
                best_mae = va_mae
                bad = 0
                best_state = {k: v.detach().cpu().clone() for k, v in hcr.state_dict().items()}
            else:
                bad += 1
                if bad >= patience:
                    break

            # skromny log co 25 epok
            if ep % 25 == 0 or improved:
                print(f"[trial {trial.number:03d} | ep {ep:4d}] val_mae={va_mae:.6f} | best={best_mae:.6f}")

        if best_state is not None:
            hcr.load_state_dict(best_state)

        # końcowa metryka walidacyjna (MAE) – to minimalizuje Optuna
        final_p_va, final_y_va = _predict_scaled(hcr, dl_va, DEVICE)
        mae_va = mean_absolute_error(final_y_va.ravel(), final_p_va.ravel())

        # sprzątanie
        del hcr, opt, sched
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        return mae_va


    # ----------------------- Uruchomienie strojenia -----------------------

    study = optuna.create_study(
        direction="minimize",
        sampler=TPESampler(
            seed=42, multivariate=True, group=True,
            n_startup_trials=80  # więcej rozgrzewki przy szerszych zakresach
        ),
        pruner=HyperbandPruner(
            min_resource=10,                # min epok zanim porównamy
            max_resource=EPOCHS_HCR,        # pełen budżet epok
            reduction_factor=3
        ),
    )


    study.optimize(objective, n_trials=3000, show_progress_bar=True, gc_after_trial=True)

    print("Best trial value (val MAE):", study.best_value)
    print("Best hyperparameters:", study.best_trial.params)


    # ==============================
    # Finalny model na najlepszych HP + TEST
    # ==============================

    best = study.best_trial.params
    DEGREE_BEST    = best.get("degree", 8)
    GRID_SIZE_BEST = best.get("grid_size", 256)
    LR_BEST        = best.get("lr", 1e-3)
    WD_BEST        = best.get("weight_decay", 1e-3)
    PATIENCE_BEST  = best.get("patience", 60)
    CLIP_BEST      = best.get("clip_grad", 1.0)
    OPT_KIND_BEST  = best.get("optimizer", "adamw")

    hcr = build_hcr_cond2d(
        degree=DEGREE_BEST,
        basis=BASIS_NAME,
        grid_size=GRID_SIZE_BEST,
        coeff_init="xavier",
        dtype=torch.float32,
        device=DEVICE,
    ).to(DEVICE)

    if OPT_KIND_BEST == "adam":
        opt = torch.optim.Adam(hcr.parameters(), lr=LR_BEST, weight_decay=WD_BEST)
    else:
        opt = torch.optim.AdamW(hcr.parameters(), lr=LR_BEST, weight_decay=WD_BEST)

    loss_fn = nn.MSELoss()
    sched   = torch.optim.lr_scheduler.ReduceLROnPlateau(
        opt, mode="min", factor=0.5, patience=max(3, PATIENCE_BEST // 4))

    best_mae = float("inf"); bad = 0; best_state = None
    min_delta = 1e-4

    for ep in range(1, EPOCHS_HCR + 1):
        # train
        hcr.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            pred = hcr(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            if CLIP_BEST and CLIP_BEST > 0.0:
                torch.nn.utils.clip_grad_norm_(hcr.parameters(), max_norm=CLIP_BEST)
            opt.step()

        # valid — MAE[-1,1]
        p_va, y_va = _predict_scaled(hcr, dl_va, DEVICE)
        va_mae = mean_absolute_error(y_va.ravel(), p_va.ravel())
        sched.step(va_mae)

        improved = (best_mae - va_mae) > min_delta
        if improved:
            best_mae = va_mae; bad = 0
            best_state = {k: v.detach().cpu().clone() for k, v in hcr.state_dict().items()}
        else:
            bad += 1

        if ep % 20 == 0 or improved:
            print(f"[HCR {ep:4d}] val_mae_scaled={va_mae:.6f} | best={best_mae:.6f}")
        if bad >= PATIENCE_BEST:
            print(f"HCR Early stop @ {ep} (patience={PATIENCE_BEST})")
            break

    if best_state is not None:
        hcr.load_state_dict(best_state)

    # --- TEST
    u1_pred_scaled, u1_true_scaled = _predict_scaled(hcr, dl_te, DEVICE)

    def mape_safe(y_true, y_pred, eps=1e-8):
        y_true = np.asarray(y_true).ravel()
        y_pred = np.asarray(y_pred).ravel()
        denom = np.maximum(np.abs(y_true), eps)
        return float(np.mean(np.abs((y_true - y_pred)/denom)))

    mae_hcr  = mean_absolute_error(u1_true_scaled, u1_pred_scaled)
    mape_hcr = mape_safe(u1_true_scaled, u1_pred_scaled)
    print(f"[HCR] {FEATURE_NAME} | TEST MAE={mae_hcr:.6f} | TEST MAPE={mape_hcr:.6f}")

    # --- Wykres HCR vs target ---
    N_test_len   = u1_true_scaled.shape[0]
    n_train_old  = len(y_train_5_days_trade_sign) if 'y_train_5_days_trade_sign' in globals() else 0
    x_axis       = np.arange(n_train_old, n_train_old + N_test_len)

    plt.figure(figsize=(10,4))
    plt.plot(x_axis, u1_true_scaled,  label='Target (H=1)', lw=2.0, color='black')
    plt.plot(x_axis, u1_pred_scaled,  label=f'HCR ({BASIS_NAME}, deg={DEGREE_BEST}, grid={GRID_SIZE_BEST})', lw=1.4)
    plt.title(f'{FEATURE_NAME} — HCR vs target (scaled)')
    plt.xlabel('Time index'); plt.ylabel('scaled value (-1..1)')
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

    class MultiHorizonDataset(Dataset):
        """
        x: (N, T, F)
        y: (N, H, F)
        """
        def __init__(self, x_np, y_np):
            x = torch.as_tensor(x_np, dtype=torch.float32)
            y = torch.as_tensor(y_np, dtype=torch.float32)
            assert x.ndim == 3 and y.ndim == 3, "Oczekuję x:(N,T,F), y:(N,H,F)"
            assert x.shape[0] == y.shape[0] and x.shape[2] == y.shape[2], "N i F muszą się zgadzać"
            self.x, self.y = x, y

        def __len__(self):
            return self.x.shape[0]

        def __getitem__(self, idx):
            return self.x[idx], self.y[idx]

    def make_loaders_mh(Xtr, Ytr, Xva, Yva, Xte, Yte, bs=BATCH_SIZE, num_workers=0, pin_memory=False):
        tr_ds = MultiHorizonDataset(Xtr, Ytr)
        va_ds = MultiHorizonDataset(Xva, Yva)
        te_ds = MultiHorizonDataset(Xte, Yte)
        tr_dl = DataLoader(tr_ds, batch_size=bs, shuffle=True,  num_workers=num_workers, pin_memory=pin_memory)
        va_dl = DataLoader(va_ds, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
        te_dl = DataLoader(te_ds, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
        return tr_dl, va_dl, te_dl

    train_dl_mh, valid_dl_mh, test_dl_mh = make_loaders_mh(X_train_mh, Y_train_mh, X_valid_mh, Y_valid_mh, X_test_mh, Y_test_mh)

    xb, yb = next(iter(train_dl_mh))
    print("Batch shapes:", tuple(xb.shape), tuple(yb.shape))  # (B, T, F), (B, H, F)


    F = len(MH_FEATURES)
    H = len(HORIZONS)

    class LSTM_MH(nn.Module):
        """
        LSTM -> linear -> (B, H, F)
        """
        def __init__(self, in_features=F, hidden_size=128, num_layers=1, dropout=0.0, horizons=H):
            super().__init__()
            self.lstm = nn.LSTM(
                input_size=in_features,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0.0
            )
            self.head = nn.Linear(hidden_size, horizons * in_features)
            self.horizons = horizons
            self.in_features = in_features

        def forward(self, x):  # x: (B, T, F)
            _, (h, _) = self.lstm(x)         # h: (num_layers, B, hidden)
            h_last = h[-1]                    # (B, hidden)
            out = self.head(h_last)           # (B, H*F)
            out = out.view(-1, self.horizons, self.in_features)  # (B, H, F)
            return out


    class AdaptiveStudent(nn.Module):
        """
        'Adaptacyjna EMA' z parametrami ucznia wyznaczanymi z okna wejściowego.
        - Uczy się `alpha` w (0,1) per FEATURE (i opcjonalnie zależne od kontekstu).
        - Generuje H prognoz rekurencyjnie: y_{t+h} = alpha * y_{t+h-1} + (1-alpha) * x_{t}
        gdzie x_t to wektor na końcu okna wejściowego (ostatni punkt).
        """
        def __init__(self, in_features=F, horizons=H, context_hidden=64):
            super().__init__()
            self.in_features = in_features
            self.horizons = horizons
            # mała sieć, która z ostatniego okna robi wektor alf
            self.phi = nn.Sequential(
                nn.Flatten(),                       # (B, T*F)
                nn.Linear(INPUT_LEN*in_features, context_hidden),
                nn.ReLU(),
                nn.Linear(context_hidden, in_features),
                nn.Sigmoid()                        # alpha w (0,1)
            )
            # waga miksu: czy używać bardziej "ostatniej obserwacji" czy "poprzedniej prognozy"
            # Nie jest konieczna, ale pomaga stabilizować.
            self.mix_gate = nn.Sequential(
                nn.Flatten(),
                nn.Linear(INPUT_LEN*in_features, in_features),
                nn.Sigmoid()
            )

        def forward(self, x):  # x: (B, T, F)
            B = x.size(0)
            last = x[:, -1, :]                   # (B, F) ostatni punkt z okna
            alpha = self.phi(x)                  # (B, F) adaptacyjne alfy
            gate  = self.mix_gate(x)             # (B, F) bramka miksująca

            preds = []
            y_prev = last
            for _ in range(self.horizons):
                # adaptacyjna ema: y_next = alpha * y_prev + (1-alpha) * last
                y_next = alpha * y_prev + (1 - alpha) * last
                # lekki mix z wejściem (można pominąć)
                y_next = gate * y_next + (1 - gate) * last
                preds.append(y_next)
                y_prev = y_next.detach()  # odpięcie BPTT w głąb horyzontu (stabilniej); usuń .detach() jeśli chcesz pełny BPTT
            y = torch.stack(preds, dim=1)         # (B, H, F)
            return y

    def mse_mh(pred, target):
        # pred, target: (B, H, F)
        return ((pred - target)**2).mean()

    @torch.no_grad()
    def evaluate_mh(model, loader):
        model.eval()
        losses = []
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pb = model(xb)
            losses.append(mse_mh(pb, yb).item())
        return float(np.mean(losses))

    def train_mh(model, train_loader, valid_loader, epochs=EPOCHS, lr=LR, patience=PATIENCE, verbose_every=10, save_path=None):
        opt = torch.optim.Adam(model.parameters(), lr=lr)
        best_val = float('inf')
        bad = 0
        history = {'train': [], 'valid': []}

        model.to(DEVICE)

        for ep in range(1, epochs+1):
            model.train()
            losses = []
            for xb, yb in train_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                opt.zero_grad()
                pb = model(xb)
                loss = mse_mh(pb, yb)
                loss.backward()
                opt.step()
                losses.append(loss.item())
            tr = float(np.mean(losses))
            va = evaluate_mh(model, valid_loader)
            history['train'].append(tr); history['valid'].append(va)

            improved = va < best_val - 1e-9
            if improved:
                best_val = va
                bad = 0
                if save_path:
                    torch.save(model.state_dict(), save_path)
            else:
                bad += 1

            if ep % verbose_every == 0 or improved:
                print(f"[{ep:4d}] train={tr:.6f} | valid={va:.6f} | best={best_val:.6f}")

            if bad >= patience:
                print(f"Early stop (patience={patience}) @ epoch {ep}")
                break

        return history, best_val

    lstm_mh = LSTM_MH(in_features=F, hidden_size=128, num_layers=1, dropout=0.0, horizons=H).to(DEVICE)
    student  = AdaptiveStudent(in_features=F, horizons=H, context_hidden=64).to(DEVICE)

    hist_student, best_student = train_mh(
        student, train_dl_mh, valid_dl_mh,
        epochs=EPOCHS, lr=LR, patience=PATIENCE,
        verbose_every=10, save_path="student_mh.pt"
    )

    hist_lstm, best_lstm = train_mh(
        lstm_mh, train_dl_mh, valid_dl_mh,
        epochs=EPOCHS, lr=LR, patience=PATIENCE,
        verbose_every=10, save_path="lstm_mh.pt"
    )

    print("Best valid: student:", best_student, " | lstm:", best_lstm)

    def ema_forecast_batch(last_window, alpha, horizons=HORIZONS):
        """
        last_window: (B, T, F)
        alpha: float w (0,1)
        Zwraca: (B, H, F)
        """
        last = last_window[:, -1, :]        # (B, F)
        y_prev = last
        preds = []
        for _ in horizons:
            y_next = alpha * y_prev + (1 - alpha) * last
            preds.append(y_next)
            y_prev = y_next
        return torch.stack(preds, dim=1)

    @torch.no_grad()
    def evaluate_ema(loader, alpha=0.8):
        losses = []
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pb = ema_forecast_batch(xb, alpha, HORIZONS)
            losses.append(mse_mh(pb, yb).item())
        return float(np.mean(losses))

    ema_val_loss = evaluate_ema(valid_dl_mh, alpha=0.8)
    ema_test_loss = evaluate_ema(test_dl_mh, alpha=0.8)
    print("EMA baseline | valid:", ema_val_loss, "| test:", ema_test_loss)

    @torch.no_grad()
    def predict_mh(model, loader):
        model.eval()
        Ys, Ps = [], []
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            pb = model(xb)              # (B, H, F)
            Ps.append(pb.cpu())
            Ys.append(yb.cpu())
        Y = torch.cat(Ys, dim=0).numpy()
        P = torch.cat(Ps, dim=0).numpy()
        return Y, P

    # Załaduj najlepsze wagi i licz metryki
    lstm_mh.load_state_dict(torch.load("lstm_mh.pt", map_location=DEVICE))
    student.load_state_dict(torch.load("student_mh.pt", map_location=DEVICE))

    Y_test_np = Y_test_mh
    Y_stud_np = predict_mh(student, test_dl_mh)[1]
    Y_lstm_np = predict_mh(lstm_mh, test_dl_mh)[1]

    # EMA
    @torch.no_grad()
    def predict_ema(loader, alpha=0.8):
        Ps = []
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            pb = ema_forecast_batch(xb, alpha, HORIZONS)
            Ps.append(pb.cpu())
        P = torch.cat(Ps, dim=0).numpy()
        return P

    Y_ema_np = predict_ema(test_dl_mh, alpha=0.8)

    def mse_per_horizon(pred, target):   # (N, H, F)
        return ((pred - target)**2).mean(axis=(0,2))  # (H,) średnia po N i F

    def mae_per_horizon(pred, target):
        return (np.abs(pred - target)).mean(axis=(0,2))

    print("MSE per H (student):", mse_per_horizon(Y_stud_np, Y_test_np))
    print("MSE per H (lstm)   :", mse_per_horizon(Y_lstm_np, Y_test_np))
    print("MSE per H (ema)    :", mse_per_horizon(Y_ema_np,  Y_test_np))

    FEATURE_TO_PLOT = 'Close'
    f_idx = MH_FEATURES.index(FEATURE_TO_PLOT)

    def dist_stats(pred_mh, target_mh, f_idx):
        """
        pred_mh: (N, H, F), target_mh: (N, H, F)
        Zwraca ramkę: średnie z H, std po H, oraz kilka kwantyli po H (po N uśredniamy)
        """
        # statystyki liczymy po horyzontach (oś=1) i po N uśredniamy
        mean_pred = pred_mh[:,:,f_idx].mean(axis=1)            # (N,)
        std_pred  = pred_mh[:,:,f_idx].std(axis=1)             # (N,)
        q10 = np.quantile(pred_mh[:,:,f_idx], 0.10, axis=1)    # (N,)
        q50 = np.quantile(pred_mh[:,:,f_idx], 0.50, axis=1)
        q90 = np.quantile(pred_mh[:,:,f_idx], 0.90, axis=1)
        # target „następny” możemy wziąć np. h=1 (pierwszy horyzont) — to typowa referencja
        tgt = target_mh[:,0,f_idx]                              # (N,)
        df = pd.DataFrame({
            "target_h1": tgt,
            "pred_mean": mean_pred,
            "pred_std": std_pred,
            "q10": q10,
            "q50": q50,
            "q90": q90
        })
        return df

    df_student = dist_stats(Y_stud_np, Y_test_np, f_idx)
    df_lstm    = dist_stats(Y_lstm_np,  Y_test_np, f_idx)
    df_ema     = dist_stats(Y_ema_np,   Y_test_np, f_idx)

    def rmse(a, b): return float(np.sqrt(np.mean((a - b)**2)))
    print("RMSE(mean) student:", rmse(df_student['pred_mean'].values, df_student['target_h1'].values))
    print("RMSE(mean) lstm   :", rmse(df_lstm['pred_mean'].values,    df_lstm['target_h1'].values))
    print("RMSE(mean) ema    :", rmse(df_ema['pred_mean'].values,     df_ema['target_h1'].values))

    # Wykres: porównanie „próbki rozkładu” dla jednego przykładu z testu
    idx = np.random.randint(0, Y_test_np.shape[0])  # wybierz losową próbkę
    x_axis = np.arange(len(HORIZONS))

    plt.figure(figsize=(8,4))
    plt.plot(x_axis, Y_test_np[idx,:,f_idx], label="Target (H)")
    plt.plot(x_axis, Y_stud_np[idx,:,f_idx], label="Student")
    plt.plot(x_axis, Y_lstm_np[idx,:,f_idx], label="LSTM")
    plt.plot(x_axis, Y_ema_np[idx,:,f_idx],  label="EMA")
    plt.xlabel("Horyzont indeks")
    plt.ylabel(f"{FEATURE_TO_PLOT} (scaled)")
    plt.title(f"Porównanie profilu prognoz po horyzontach (próbka {idx})")
    plt.legend()
    plt.show()

    # Wykres: „rozkład” (histogram) KILKU próbek po H dla danego modelu
    def plot_empirical_distribution(P, label):
        vals = P[:, :, f_idx].reshape(-1)    # zbierz wszystkie horyzonty i próbki
        plt.figure(figsize=(6,4))
        plt.hist(vals, bins=40, alpha=0.8, density=True)
        plt.title(f"Empiryczny rozkład prognoz ({label}) — {FEATURE_TO_PLOT}")
        plt.xlabel("wartość (scaled)"); plt.ylabel("gęstość")
        plt.show()

    plot_empirical_distribution(Y_stud_np, "Adaptive Student")
    plot_empirical_distribution(Y_lstm_np, "LSTM")
    plot_empirical_distribution(Y_ema_np,  "EMA")

    def mse_per_hf(pred, target):  # (N,H,F)
        # MSE osobno dla par (h, f)
        return ((pred - target)**2).mean(axis=0)  # (H, F)

    mse_student_hf = mse_per_hf(Y_stud_np, Y_test_np)
    mse_lstm_hf    = mse_per_hf(Y_lstm_np,  Y_test_np)
    mse_ema_hf     = mse_per_hf(Y_ema_np,   Y_test_np)

    df_mse_student = pd.DataFrame(mse_student_hf, columns=MH_FEATURES, index=[f"H{h}" for h in HORIZONS])
    df_mse_lstm    = pd.DataFrame(mse_lstm_hf,    columns=MH_FEATURES, index=[f"H{h}" for h in HORIZONS])
    df_mse_ema     = pd.DataFrame(mse_ema_hf,     columns=MH_FEATURES, index=[f"H{h}" for h in HORIZONS])

    print("MSE per (H,F) — Student")
    display(df_mse_student.style.format("{:.6f}"))

    print("MSE per (H,F) — LSTM")
    display(df_mse_lstm.style.format("{:.6f}"))

    print("MSE per (H,F) — EMA")
    display(df_mse_ema.style.format("{:.6f}"))

    def pick_horizon(Y, h_index: int):
        """
        Y: (N, H, F), zwraca (N, F) dla wybranego horyzontu.
        h_index = 0 oznacza pierwszy krok naprzód.
        """
        return Y[:, h_index, :]

    def inverse_feature(arr_2d, feature_index, scaler):
        """
        arr_2d: (N, F) w skali scaler_mh (-1,1)
        Zwraca wektor (N,) dla jednej cechy w skali oryginalnej.
        """
        N, F = arr_2d.shape
        tmp = np.zeros((N, F))
        tmp[:] = scaler.min_ / (1 - (-1)) * 0  # placeholder; i tak nadpiszemy cechę
        tmp[:, feature_index] = arr_2d[:, feature_index]
        inv = scaler.inverse_transform(tmp)
        return inv[:, feature_index]

    def mae_mape(pred, true, eps=1e-8):
        """
        MAE i MAPE z ochroną przed dzieleniem przez 0 (MAPE).
        pred, true: (N,) lub (N,F) (wtedy liczona po wszystkich elementach).
        """
        pred = np.asarray(pred); true = np.asarray(true)
        mae = mean_absolute_error(true.ravel(), pred.ravel())
        # MAPE: ochronimy się przed 0 dodając eps do mianownika
        denom = np.maximum(np.abs(true.ravel()), eps)
        mape = np.mean(np.abs((true.ravel() - pred.ravel()) / denom))
        return mae, mape

    ## Comparision new methods

    # Rysujemy np. 'Close' = idx=1 (Open=0, Close=1, High=2, Low=3, trade_sign=4)
    feature_name = 'close'
    idx = MH_FEATURES.index(feature_name)

    # Wybieramy pierwszy horyzont (H=1 => h_index=0)
    y_test_h1   = pick_horizon(Y_test_np, 0)       # (N, F)
    y_stud_h1   = pick_horizon(Y_stud_np, 0)
    y_lstm_h1   = pick_horizon(Y_lstm_np, 0)
    y_ema_h1    = pick_horizon(Y_ema_np,  0)

    # Oś czasu (doklejony do końca trainu, jak w Twoim przykładzie):
    n_train_old = len(y_train_5_days_trade_sign) if 'y_train_5_days_trade_sign' in globals() else 0
    x_axis = np.arange(n_train_old, n_train_old + y_test_h1.shape[0])

    plt.figure(figsize=(10,4))
    plt.plot(x_axis, y_test_h1[:, idx],  label='target (H=1)', lw=1.5)
    plt.plot(x_axis, y_stud_h1[:, idx],  label='Adaptive Student', lw=1)
    plt.plot(x_axis, y_lstm_h1[:, idx],  label='LSTM-MH', lw=1)
    plt.plot(x_axis, y_ema_h1[:, idx],   label='EMA baseline', lw=1)
    plt.title(f'{feature_name} — prognoza jednego kroku (H=1) — skala znormalizowana')
    plt.xlabel('time index'); plt.ylabel('scaled value (-1..1)')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # MAE/MAPE (w skali znormalizowanej)
    for name, pred in [('Student', y_stud_h1), ('LSTM-MH', y_lstm_h1), ('EMA', y_ema_h1)]:
        mae, mape = mae_mape(pred[:, idx], y_test_h1[:, idx])
        print(f"[Scaled] {name:9s} | MAE={mae:.6f} | MAPE={mape:.6f}")

    ## Final Compariosn (yet without HCR-NN)
    # =======================
    # Final Comparison Figure (z HCR)
    # =======================

    # ---- Config ----
    feature_name = 'Close'  # zmień na dowolne z MH_FEATURES
    f_idx = MH_FEATURES.index(feature_name)
    h_index = 0             # H=1 => index 0
    title = f"One-step Ahead Forecast (H=1) — {feature_name} (scaled)"

    # ---- Helpers ----
    def pick_horizon(Y, h_index: int):
        return Y[:, h_index, :]  # (N, F)

    def mape_safe(y_true, y_pred, eps=1e-8):
        y_true = np.asarray(y_true).ravel()
        y_pred = np.asarray(y_pred).ravel()
        denom = np.maximum(np.abs(y_true), eps)
        return float(np.mean(np.abs((y_true - y_pred) / denom)))

    # ---- Data: targets + predictions (scaled) ----
    # Multi-horizon targets/preds
    y_test_h1 = pick_horizon(Y_test_np, h_index)          # (N, F)
    series = []  # (label, values_1d)

    # Adaptive Student
    y_stud_h1 = pick_horizon(Y_stud_np, h_index)
    mae_s  = mean_absolute_error(y_test_h1[:, f_idx], y_stud_h1[:, f_idx])
    mape_s = mape_safe(y_test_h1[:, f_idx], y_stud_h1[:, f_idx])
    series.append((f"Adaptive Student (MAE={mae_s:.4f}, MAPE={mape_s:.2%})", y_stud_h1[:, f_idx]))

    # LSTM-MH
    y_lstm_h1 = pick_horizon(Y_lstm_np, h_index)
    mae_l  = mean_absolute_error(y_test_h1[:, f_idx], y_lstm_h1[:, f_idx])
    mape_l = mape_safe(y_test_h1[:, f_idx], y_lstm_h1[:, f_idx])
    series.append((f"LSTM-MH (MAE={mae_l:.4f}, MAPE={mape_l:.2%})", y_lstm_h1[:, f_idx]))

    # EMA baseline
    y_ema_h1 = pick_horizon(Y_ema_np, h_index)
    mae_e  = mean_absolute_error(y_test_h1[:, f_idx], y_ema_h1[:, f_idx])
    mape_e = mape_safe(y_test_h1[:, f_idx], y_ema_h1[:, f_idx])
    series.append((f"EMA baseline (MAE={mae_e:.4f}, MAPE={mape_e:.2%})", y_ema_h1[:, f_idx]))

    have_single_step = ('y_pred_np' in globals()) and ('y_test_np' in globals()) and (y_test_np.ndim == 2)
    if have_single_step:
        idx_old = 1  # Close w starej konwencji
        mae_o  = mean_absolute_error(y_test_np[:, idx_old], y_pred_np[:, idx_old])
        mape_o = mape_safe(y_test_np[:, idx_old], y_pred_np[:, idx_old])
        series.append((f"Single-step LSTM (MAE={mae_o:.4f}, MAPE={mape_o:.2%})", y_pred_np[:, idx_old]))

    # ---- X-axis aligned with earlier plots ----
    n_train_old = len(y_train_5_days_trade_sign) if 'y_train_5_days_trade_sign' in globals() else 0
    N_test_len = y_test_h1.shape[0]
    x_axis = np.arange(n_train_old, n_train_old + N_test_len)


    have_hcr = ('u1_pred_scaled' in globals()) and ('u1_true_scaled' in globals())
    if have_hcr:
        L = min(N_test_len, len(u1_pred_scaled))
        mae_h  = mean_absolute_error(y_test_h1[:L, f_idx], u1_pred_scaled[:L])
        mape_h = mape_safe(y_test_h1[:L, f_idx],           u1_pred_scaled[:L])

        basis_label = BASIS_NAME if 'BASIS_NAME' in globals() else 'cosine'
        deg_label   = DEGREE if 'DEGREE' in globals() else None
        label = f"HCR ({basis_label}" + (f", deg={deg_label}" if deg_label is not None else "") \
                + f") (MAE={mae_h:.4f}, MAPE={mape_h:.2%})"
        series.append((label, u1_pred_scaled[:L]))
    else:
        print("[INFO] HCR: nie znaleziono u1_pred_scaled/u1_true_scaled. Uruchom wcześniej blok HCR (Twoja biblioteka).")

    # ---- Plot ----
    plt.figure(figsize=(12, 5))
    # target (scaled)
    plt.plot(x_axis, y_test_h1[:, f_idx], label="Target (H=1)", linewidth=2.0)

    # modele
    linestyles = ['-', '--', '-.', ':']  # rotacja stylów
    for i, (lbl, vals) in enumerate(series):
        if len(vals) != N_test_len:
            L = min(len(vals), N_test_len)
            plt.plot(x_axis[:L], vals[:L], label=lbl, linestyle=linestyles[i % len(linestyles)], linewidth=1.4)
        else:
            plt.plot(x_axis, vals, label=lbl, linestyle=linestyles[i % len(linestyles)], linewidth=1.4)

    plt.title(title)
    plt.xlabel("Time index")
    plt.ylabel("Scaled value (-1..1)")
    plt.grid(alpha=0.25)
    plt.legend(loc="best", frameon=True)
    plt.tight_layout()
    plt.show()

    # ---- Compact metrics table ----
    print("=== Scaled MAE/MAPE on H=1 —", feature_name, "===")
    print(f"Adaptive Student: MAE={mae_s:.6f}, MAPE={mape_s:.6f}")
    print(f"LSTM-MH        : MAE={mae_l:.6f}, MAPE={mape_l:.6f}")
    print(f"EMA baseline   : MAE={mae_e:.6f}, MAPE={mape_e:.6f}")
    if have_single_step:
        print(f"Single-step LSTM: MAE={mae_o:.6f}, MAPE={mape_o:.6f}")
    if have_hcr:
        print(f"HCR ({basis_label}, deg={deg_label}): MAE={mae_h:.6f}, MAPE={mape_h:.6f}")


DataError: No numeric types to aggregate